<a href="https://colab.research.google.com/github/sholtodouglas/learning_from_play/blob/master/notebooks/train_lfp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb -q
!pip install pathy -q
!pip install comet_ml -q

     |████████████████████████████████| 2.0MB 6.7MB/s 
     |████████████████████████████████| 163kB 28.9MB/s 
     |████████████████████████████████| 133kB 23.2MB/s 
     |████████████████████████████████| 102kB 8.1MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
     |████████████████████████████████| 122kB 9.5MB/s 
     |████████████████████████████████| 256kB 6.2MB/s 
     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 61kB 5.4MB/s 
     |████████████████████████████████| 522kB 8.3MB/s 


In [2]:
# import comet_ml at the top of your file
from comet_ml import Experiment
import wandb
wandb.login()


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [40]:
import argparse

parser = argparse.ArgumentParser(description='LFP training arguments')
parser.add_argument('run_name')
parser.add_argument('--train_datasets', nargs='+', help='Training dataset names')
parser.add_argument('--test_datasets', nargs='+', help='Testing dataset names')
parser.add_argument('-c', '--colab', default=False, action='store_true', help='Enable if using colab environment')
parser.add_argument('-s', '--data_source', default='DRIVE', help='Source of training data')
parser.add_argument('-tfr', '--from_tfrecords', default=False, action='store_true', help='Enable if using tfrecords format')
parser.add_argument('-d', '--device', default='TPU', help='Hardware device to train on')
parser.add_argument('-b', '--batch_size', default=512, type=int)
parser.add_argument('-wmax', '--window_size_max', default=40, type=int)
parser.add_argument('-wmin', '--window_size_min', default=20, type=int)
parser.add_argument('-la', '--actor_layer_size', default=2048, type=int, help='Layer size of actor, increases size of neural net')
parser.add_argument('-le', '--encoder_layer_size', default=512, type=int, help='Layer size of encoder, increases size of neural net')
parser.add_argument('-lp', '--planner_layer_size', default=2048, type=int, help='Layer size of planner, increases size of neural net')
parser.add_argument('-embd', '--img_embedding_size', default=64, type=int, help='Embedding size of features,goal space')
parser.add_argument('-z', '--latent_dim', default=256, type=int, help='Size of the VAE latent space')
parser.add_argument('-g', '--gcbc', default=False, action='store_true', help='Enables GCBC, a simpler model with no encoder/planner')
parser.add_argument('-n', '--num_distribs', default=None, type=int, help='Number of distributions to use in logistic mixture model')
parser.add_argument('-q', '--qbits', default=None, type=int, help='Number of quantisation bits to discrete distributions into. Total quantisations = 2**qbits')
parser.add_argument('-lr', '--learning_rate', type=float, default=3e-4)
parser.add_argument('-t', '--train_steps', type=int, default=200000)
parser.add_argument('-r', '--resume', default=False, action='store_true')
parser.add_argument('-B', '--beta', type=float, default=0.00003)
parser.add_argument('-i', '--images', default=False, action='store_true')
parser.add_argument('-vq', '--discrete', default=False, action='store_true')
parser.add_argument('--fp16', default=False, action='store_true')
parser.add_argument('--bucket_name', help='GCS bucket name to stream data from')
parser.add_argument('--tpu_name', help='GCP TPU name') # Only used in the script on GCP


# ## Sample colab config
# args = parser.parse_args('''
# refactor_test
# --train_dataset UR5 UR5_slow_gripper UR5_high_transition
# --test_dataset UR5_slow_gripper_test
# -c
# -tfr
# -s GCS
# -d TPU
# -b 512
# -la 2048
# -le 512
# -lp 512
# -z 256
# -lr 3e-4
# --bucket_name iowa_bucket_lfp
# '''.split())



# ## Sample colab config
# args = parser.parse_args('''
# PB0_02
# --train_dataset UR5 UR5_slow_gripper UR5_high_transition
# --test_dataset UR5_slow_gripper_test
# -c
# -s GCS
# -d TPU
# -b 16
# -la 2048
# -le 512
# -lp 2048
# -wmax 40
# -wmin 20
# -z 256
# -lr 3e-4
# -B 0.02
# -i 
# -tfr
# -n 5
# --bucket_name iowa_bucket_lfp
# '''.split())


args = parser.parse_args('''
discrete4096
--train_dataset UR5 UR5_slow_gripper UR5_high_transition
--test_dataset UR5_slow_gripper_test
-c
-tfr
-s GCS
-d TPU
-b 8
-la 2048
-le 512
-lp 8
-z 4096
-lr 3e-4
-B 0.0
--bucket_name iowa_bucket_lfp
-vq
'''.split())

# -n 5
# -q 8

print(args)

Namespace(actor_layer_size=2048, batch_size=8, beta=0.0, bucket_name='iowa_bucket_lfp', colab=True, data_source='GCS', device='TPU', discrete=True, encoder_layer_size=512, fp16=False, from_tfrecords=True, gcbc=False, images=False, img_embedding_size=64, latent_dim=4096, learning_rate=0.0003, num_distribs=None, planner_layer_size=8, qbits=None, resume=False, run_name='discrete4096', test_datasets=['UR5_slow_gripper_test'], tpu_name=None, train_datasets=['UR5', 'UR5_slow_gripper', 'UR5_high_transition'], train_steps=200000, window_size_max=40, window_size_min=20)


## Init

In [4]:
from pathlib import Path
from pathy import Pathy
import os
import requests
import json
import pprint
import logging
import numpy as np
import tensorflow as tf
import time



pp = pprint.PrettyPrinter(indent=4)
# In[4]:


#@title Workpace Setup (Local vs Colab)

# Set up working directory and libraries
if args.colab:
    from google.colab import drive, auth
    print('Using colab setup')
    WORKING_PATH = Path('/content/learning_from_play')
    # Clone repo
    try:
        get_ipython().system("git clone 'https://github.com/sholtodouglas/learning_from_play' {WORKING_PATH}")
    except: 
        pass
    # Mount drive
    drive.mount('/content/drive')
else:
    print('Using local setup')
    WORKING_PATH = Path.cwd()
    print(f'Working path: {WORKING_PATH}')

# Change working directory to learning_from_play
os.chdir(WORKING_PATH)
import lfp

# Set up storage directory and datasets
if args.data_source == 'DRIVE':
    assert args.colab, "Must be using Colab"
    print('Reading data from Google Drive')
    STORAGE_PATH = Path('/content/drive/My Drive/Robotic Learning')
elif args.data_source == 'GCS':
    if args.colab:
      auth.authenticate_user()
    print('Reading data from Google Cloud Storage')
    r = requests.get('https://ipinfo.io')
    region = r.json()['region']
    project_id = 'learning-from-play-303306'
    logging.warning(f'You are accessing GCS data from {region}, make sure this is the same as your bucket {args.bucket_name}')
    STORAGE_PATH = Pathy(f'gs://{args.bucket_name}')
else:
    print('Reading data from local filesystem')
    STORAGE_PATH = WORKING_PATH

print(f'Storage path: {STORAGE_PATH}')
TRAIN_DATA_PATHS = [STORAGE_PATH/'data'/x for x in args.train_datasets]
TEST_DATA_PATHS = [STORAGE_PATH/'data'/x for x in args.test_datasets]

Using colab setup
Cloning into '/content/learning_from_play'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (316/316), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 2796 (delta 207), reused 149 (delta 75), pack-reused 2480
Receiving objects: 100% (2796/2796), 126.34 MiB | 22.01 MiB/s, done.
Resolving deltas: 100% (1685/1685), done.
Mounted at /content/drive
No pybullet installation found - which is fine if training


Reading data from Google Cloud Storage
Storage path: gs://iowa_bucket_lfp/


In [6]:

print("Tensorflow version " + tf.__version__)

if args.device == 'TPU':
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=args.tpu_name)  # TPU detection
        print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    except ValueError:
        raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    NUM_DEVICES = strategy.num_replicas_in_sync
    print("REPLICAS: ", NUM_DEVICES)
    if args.fp16:
        tf.keras.mixed_precision.set_global_policy('mixed_bfloat16')
else:
    physical_devices = tf.config.list_physical_devices()
    if args.device == 'GPU':
        tf.config.experimental.set_memory_growth(physical_devices[3], enable=True)
        if args.fp16:
            tf.keras.mixed_precision.set_global_policy('mixed_float16')
    strategy = tf.distribute.get_strategy()
    NUM_DEVICES = 1
    print(physical_devices)

Tensorflow version 2.4.1
Running on TPU  ['10.12.16.10:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.12.16.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.12.16.10:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [37]:
# Use this to edit modules without needing to restart the kernel (can also edit local, push/pull)
!git pull
import importlib
importlib.reload(lfp.data)
importlib.reload(lfp.model)
importlib.reload(lfp.train)
importlib.reload(lfp.metric)

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/sholtodouglas/learning_from_play
   f60e28a..4ba8d1e  master     -> origin/master
Updating f60e28a..4ba8d1e
Fast-forward
 lfp/train.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


<module 'lfp.metric' from '/content/learning_from_play/lfp/metric.py'>

## Data

In [11]:
GLOBAL_BATCH_SIZE = args.batch_size * NUM_DEVICES
dl = lfp.data.PlayDataloader(include_imgs = args.images, batch_size=GLOBAL_BATCH_SIZE,  window_size=args.window_size_max, min_window_size=args.window_size_min)

In [12]:
# Train data
train_data = dl.extract(TRAIN_DATA_PATHS, from_tfrecords=args.from_tfrecords)
train_dataset = dl.load(train_data)

{   'acts': TensorSpec(shape=(64, None, 7), dtype=tf.float32, name=None),
    'dataset_path': TensorSpec(shape=(64, None), dtype=tf.int32, name=None),
    'goals': TensorSpec(shape=(64, None, 11), dtype=tf.float32, name=None),
    'obs': TensorSpec(shape=(64, None, 18), dtype=tf.float32, name=None),
    'tstep_idxs': TensorSpec(shape=(64, None), dtype=tf.int32, name=None)}


In [13]:
# Validation data
valid_data = dl.extract(TEST_DATA_PATHS, from_tfrecords=args.from_tfrecords)
valid_dataset = dl.load(valid_data)

{   'acts': TensorSpec(shape=(64, None, 7), dtype=tf.float32, name=None),
    'dataset_path': TensorSpec(shape=(64, None), dtype=tf.int32, name=None),
    'goals': TensorSpec(shape=(64, None, 11), dtype=tf.float32, name=None),
    'obs': TensorSpec(shape=(64, None, 18), dtype=tf.float32, name=None),
    'tstep_idxs': TensorSpec(shape=(64, None), dtype=tf.int32, name=None)}


# Training Loop

In [41]:
from lfp.train import LFPTrainer

def train_setup():
    model_params = {'obs_dim':args.img_embedding_size + dl.proprioceptive_features_dim if args.images else dl.obs_dim,
                'goal_dim':args.img_embedding_size if args.images else dl.goal_dim,
                'act_dim':dl.act_dim,
                'layer_size':args.actor_layer_size, 
                'latent_dim':args.latent_dim}

    actor = lfp.model.create_actor(**model_params, gcbc=args.gcbc, num_distribs=args.num_distribs, qbits=args.qbits)

    if args.gcbc:
        encoder = None
        planner = None
    else:
        
        model_params['layer_size'] = args.encoder_layer_size
        if args.discrete:
          encoder = lfp.model.create_discrete_encoder(**model_params)
        else:
          encoder = lfp.model.create_encoder(**model_params)

        model_params['layer_size'] = args.planner_layer_size
        planner = lfp.model.create_planner(**model_params)

    if args.images:
      cnn = lfp.model.cnn(dl.img_size, dl.img_size, embedding_size=args.img_embedding_size)
      lfp.utils.build_cnn(cnn)  # Have to do this becasue it is subclassed and the reshapes in the spatial softmax don't play nice with model auto build
    else:
      cnn = None

    #optimizer = tfa.optimizers.LAMB(learning_rate=args.learning_rate)
    optimizer = optimizer = tf.optimizers.Adam
    trainer = LFPTrainer(args, actor, dl, encoder, planner, cnn, optimizer, strategy, GLOBAL_BATCH_SIZE)
    trainer.temperature = 1./16.
    return actor, encoder, planner, cnn, trainer


if args.device=='CPU' or args.device=='GPU':
     actor, encoder, planner, cnn, trainer = train_setup()
else:
    with strategy.scope():
         actor, encoder, planner, cnn, trainer = train_setup()
        
        
train_dist_dataset = iter(strategy.experimental_distribute_dataset(train_dataset))
valid_dist_dataset = iter(strategy.experimental_distribute_dataset(valid_dataset))
plotting_dataset = iter(valid_dataset) #for the cluster fig, easier with a non distributed dataset


In [42]:

from tensorflow.keras.utils import Progbar
progbar = Progbar(args.train_steps, verbose=1, interval=0.5)
valid_inc = 20
save_inc = 2000

In [43]:
run_name = args.run_name
model_path = str(STORAGE_PATH/'saved_models'/args.run_name)

if args.resume:
  # WandB reinit
  with open(f'{model_path}/config.json', 'r') as f:
      data = json.load(f)
  # Comet ML reinit
  exp = ExistingExperiment(api_key="C4vcCM57bnSYEsdncguxDW8pO",  previous_experiment=data['experiment_key'])

  wandb.init(project="learning-from-play_v2", id=data['run_id'],  resume="must")
  t = wandb.run.step + valid_inc # Todo get this from comet to complete the transition

  load_weights(model_path, actor, encoder, planner, with_optimizer=True)
  print('Loaded model weights and optimiser state')
  
  prognar.add(t, []) # update the progbar to the most recent point
else:
  #Comet
  experiment = Experiment(api_key="C4vcCM57bnSYEsdncguxDW8pO",project_name="learning-from-play",workspace="sholtodouglas")
  experiment.set_name(run_name)
  # WandB
  wandb.init(project="learning-from-play_v2")
  wandb.run.name = run_name
  t = 0


COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/sholtodouglas/learning-from-play/ca44f3a3e31640d99908e881db1a26ae
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     actor_grad_norm [100]          : (0.025230014696717262, 0.08431269973516464)
COMET INFO:     encoder_grad_norm [100]        : (0.0005770906573161483, 0.011739377863705158)
COMET INFO:     global_grad_norm [100]         : (0.025299275293946266, 0.08543957024812698)
COMET INFO:     planner_grad_norm [100]        : 100
COMET INFO:     step_time [100]                : (0.5, 83.5)
COMET INFO:     train_act_with_enc_loss [100]  : (0.002833110745996237, 0.03515952080488205)
COMET INFO:     train_act_with_plan_loss [100] : 100
COMET INFO:     train_loss [100]               : (0.002833110745996237, 0.03515952080488205)
COMET INFO:    

train_loss,0.00283
valid_loss,0.00368
actor_grad_norm,0.03006
encoder_grad_norm,0.00141
planner_grad_norm,0.0
global_grad_norm,0.03013
train_act_with_enc_loss,0.00283
train_act_with_plan_loss,0.0
valid_act_with_enc_loss,0.00368
valid_act_with_plan_loss,0.0
train_reg_loss,0.0


train_loss,█▃▂▂▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
actor_grad_norm,█▃▃▃▆██▄▆▅▅▄▃▂▂▂▂▂▂▂▂▄▂▂▂▂▂▂▃▂▂▃▂▁▁▁▁▁▁▂
encoder_grad_norm,▁▁▃▃▄█▇▃▄▅▃▆▅▄▄▅▄▃▃▂▃▃▄▃▃▃▃▂▃▃▂▃▃▂▂▃▂▂▂▂
planner_grad_norm,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_grad_norm,█▃▃▃▆██▄▆▅▅▄▃▂▂▂▂▂▂▂▂▄▂▂▂▂▂▂▃▂▂▃▂▁▁▂▁▁▁▂
train_act_with_enc_loss,█▃▂▂▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_act_with_plan_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_act_with_enc_loss,█▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_act_with_plan_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_reg_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [44]:
from lfp.plotting import produce_cluster_fig, project_enc_and_plan, plot_to_image
from lfp.metric import log # gets state and clears simultaneously

In [45]:
import time


In [ ]:


# Creating these autograph wrappers so that tf.data operations are executed in graph mode
#@tf.function
def train(train_dataset, beta):
    train_batch = next(train_dataset)
    trainer.distributed_train_step(train_batch, beta)

#@tf.function
def test(valid_dataset, beta):
    valid_batch = next(valid_dataset)
    trainer.distributed_test_step(valid_batch, beta)

while t < args.train_steps:
    start_time = time.time()
    train(train_dist_dataset, args.beta)

    if t % valid_inc == 0:
        test(valid_dist_dataset, args.beta)
        step_time = round(time.time() - start_time, 1)

        metrics = {metric_name: log(metric) for metric_name, metric in trainer.metrics.items()}
        metrics['step_time'] = step_time

        # validation plotting
        progbar.add(valid_inc, [('Train Loss', metrics['train_loss']),
                                ('Validation Loss', metrics['valid_loss']),
                                ('Time (s)', step_time)])
        #Plot on Comet
        experiment.log_metrics(metrics,step=t)
        # Plot on WandB
        wandb.log(metrics, step=t)

    if (t+1) % save_inc == 0:
        trainer.save_weights(model_path, run_id=wandb.run.id, experiment_key=experiment.get_key())
        if not args.gcbc and not args.images and not args.discrete:
          z_enc, z_plan = produce_cluster_fig(next(plotting_dataset), encoder, planner, TEST_DATA_PATHS[0], num_take=dl.batch_size//4)

          #Comet
          experiment.log_figure('z_enc', z_enc, step=t)
          experiment.log_figure('z_plan', z_plan,step=t)

          # WandB
          wandb.log({'z_enc':z_enc, 'z_plan':z_plan}, step=t)

          #latent_fig = project_enc_and_plan(ze, zp)
          #latent_img = plot_to_image(latent_fig)

    t += 1

   220/200000 [..............................] - ETA: 39:11:28 - Train Loss: 0.0133 - Validation Loss: 0.0127 - Time (s): 7.8636

In [ ]:
from tqdm import tqdm
t = time.time()
for i in tqdm(range(0,10)):
  train_batch = next(train_dist_dataset)
  trainer.distributed_train_step(train_batch, args.beta)
print(time.time() - t)

100%|██████████| 10/10 [00:03<00:00,  2.89it/s]

3.469046115875244
